Encuentra las cinco ciudades asiáticas más pobladas a partir de la siguiente base de datos. ¿Cuál es el idioma más hablado en cada una de ellas?

In [15]:
import pymysql
import pandas as pd
database_host = 'relational.fit.cvut.cz'
username = 'guest'
password = 'relational'
database_name = 'world'
db = pymysql.connect(host=database_host,
                     user=username,
                     password=password,
                     database=database_name)

In [16]:
pd.read_sql('SHOW TABLES',db)


C:\Users\Victor\AppData\Local\Temp\ipykernel_13348\4293925020.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('SHOW TABLES',db)


,Tables_in_world
0,City
1,Country
2,CountryLanguage


In [7]:
# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = db.cursor()

In [8]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [22]:
query = '''SELECT Code, Continent
        FROM Country
        '''
sql_query(query)

,Code,Continent
0,ABW,North America
1,AFG,Asia
2,AGO,Africa
3,AIA,North America
4,ALB,Europe
...,...,...
234,YEM,Asia
235,YUG,Europe
236,ZAF,Africa
237,ZMB,Africa


In [70]:
query = '''
SELECT *
FROM (SELECT City.Name, City.CountryCode, City.Population, Country.Continent
        FROM City
        JOIN Country
        ON City.CountryCode = Country.Code
        WHERE Continent='Asia'
        ORDER BY Population DESC
        LIMIT 5
        )
JOIN
    (SELECT Name, Language, IsOfficial, Percentage
    FROM CountryLanguage
    JOIN Country
    ON CountryLanguage.CountryCode = Country.Code
    WHERE Percentage = (SELECT MAX(Percentage))
        '''
sql_query(query)

,Name,CountryCode,Population,Continent
0,Mumbai (Bombay),IND,10500000,Asia
1,Seoul,KOR,9981619,Asia
2,Shanghai,CHN,9696300,Asia
3,Jakarta,IDN,9604900,Asia
4,Karachi,PAK,9269265,Asia


In [28]:
query = '''SELECT *
        FROM CountryLanguage
        '''
sql_query(query)

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3
1,ABW,English,F,9.5
2,ABW,Papiamento,F,76.7
3,ABW,Spanish,F,7.4
4,AFG,Balochi,F,0.9
...,...,...,...,...
979,ZMB,Tongan,F,11.0
980,ZWE,English,T,2.2
981,ZWE,Ndebele,F,16.2
982,ZWE,Nyanja,F,2.2


In [65]:
query = '''SELECT CountryCode, Language,
        (SELECT MAX(Percentage) AS MainLanguage)
        FROM CountryLanguage 
        GROUP BY DISTINCT CountryCode, Language
        WHERE MainLanguage
        '''
sql_query(query)

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'DISTINCT CountryCode, Language\n        WHERE MainLanguage' at line 4")

In [71]:
query = '''
SELECT CityName, CountryName,Population,Language,Percentage
FROM(
        SELECT City.Name AS CityName,
               Country.Name AS CountryName,
               City.Population
        FROM City
        JOIN Country
        ON City.CountryCode = Country.Code
        WHERE Continent = 'Asia'
        ORDER BY City.Population DESC
        LIMIT 5
    )T1
JOIN
        (SELECT Name, Language, IsOfficial, Percentage
        FROM CountryLanguage
        JOIN Country
        ON CountryLanguage.CountryCode = Country.Code
        WHERE Percentage = (SELECT MAX(Percentage)
                            FROM CountryLanguage
                            WHERE Country.Code = CountryLanguage.CountryCode
                            )
)T2
ON T1.CountryName = T2.Name
ORDER BY Population DESC
'''
sql_query(query)

,CityName,CountryName,Population,Language,Percentage
0,Mumbai (Bombay),India,10500000,Hindi,39.9
1,Seoul,South Korea,9981619,Korean,99.9
2,Shanghai,China,9696300,Chinese,92.0
3,Jakarta,Indonesia,9604900,Javanese,39.4
4,Karachi,Pakistan,9269265,Punjabi,48.2
